In [1]:
import cv2
from imutils import contours
import numpy as np
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [22]:
# Load image, grayscale, and adaptive threshold
image = cv2.imread('puzzle4.JPG')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 0)
thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,57,5)

cv2.imshow('thresh', thresh)


In [23]:
# Retrieve largest area in the image
cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

max_area = 0
c = 0
for i in cnts:
        area = cv2.contourArea(i)
        if area > 1000:
                if area > max_area:
                    max_area = area
                    best_cnt = i
                    image = cv2.drawContours(image, cnts, c, (0, 255, 0), 3)
        c+=1

In [24]:
mask = np.zeros((thresh.shape),np.uint8)
cv2.drawContours(mask,[best_cnt],0,255,-1)
cv2.drawContours(mask,[best_cnt],0,0,2)
cv2.imshow("mask", mask)

In [25]:
out = np.zeros_like(thresh)
out[mask == 255] = gray[mask == 255]
cv2.imshow("New image", out)

In [26]:
approx = cv2.approxPolyDP(best_cnt, 0.02 * cv2.arcLength(best_cnt, True), True)
print(len(approx))

4


In [27]:
src = np.squeeze(approx, axis=1)
src = np.array(src,np.float32)
print(src)
src.shape

[[108.  71.]
 [107. 572.]
 [607. 572.]
 [608.  72.]]


(4, 2)

In [28]:
testImg = out.copy()
cv2.imshow("test",testImg)

In [ ]:
#for x in range(4):
    #cv2.circle(testImg,(src[x][0],src[x][1]),10,(0,255,0),cv2.FILLED)

#cv2.imshow("test",testImg)

In [29]:
def width_and_height(approx):
    x_coord = list(approx[:,0][:,0])
    y_coord = list(approx[:,0][:,1])
    width = max(x_coord) - min(x_coord)
    height = max(y_coord) - min(y_coord)

    return width, height

In [30]:
width, height = width_and_height(approx)
print(width)
print(height)

501
501


In [31]:
warp_points = np.float32([[0,0],[0,height],[width,height],[width,0]])
print(warp_points)

[[  0.   0.]
 [  0. 501.]
 [501. 501.]
 [501.   0.]]


In [32]:
matrix = cv2.getPerspectiveTransform(src,warp_points)
print(matrix)

[[ 9.99563930e-01  1.99513758e-03 -1.08094559e+02]
 [-1.99911986e-03  9.99559931e-01 -7.07528502e+01]
 [-3.99025921e-06 -1.59451395e-08  1.00000000e+00]]


In [33]:
output = cv2.warpPerspective(testImg, matrix, (width,height))

In [34]:
cv2.imshow("test",output)